In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import requests
import pandas as pd
# from pandasgui import show
from alpha_vantage.alphavantage import AlphaVantage
from alpha_vantage.timeseries import TimeSeries
from bs4 import BeautifulSoup
import time
import numpy as np

def display_all(df):
    with pd.option_context("display.max_rows", 100000, "display.max_columns", 100000): 
        display(df)

# ///////////////////////////////////////////////////////////
# FUNCTIONS FOR ALPHA VANTAGE
# ///////////////////////////////////////////////////////////
TIME_SERIES_INTRADAY = 'TIME_SERIES_INTRADAY'
TIME_SERIES_INTRADAY_EXTENDED = 'TIME_SERIES_INTRADAY_EXTENDED'
TIME_SERIES_DAILY = 'TIME_SERIES_DAILY'
TIME_SERIES_DAILY_ADJUSTED = 'TIME_SERIES_DAILY_ADJUSTED'
TIME_SERIES_WEEKLY = 'TIME_SERIES_WEEKLY'
TIME_SERIES_WEEKLY_ADJUSTED = 'TIME_SERIES_WEEKLY_ADJUSTED'
TIME_SERIES_MONTHLY = 'TIME_SERIES_MONTHLY'
TIME_SERIES_MONTHLY_ADJUSTED = 'TIME_SERIES_MONTHLY_ADJUSTED'
GLOBAL_QUOTE = 'GLOBAL_QUOTE'
SYMBOL_SEARCH = 'SYMBOL_SEARCH'
OVERVIEW = 'OVERVIEW'
EARNINGS = 'EARNINGS'
INCOME_STATEMENT = 'INCOME_STATEMENT'
BALANCE_SHEET = 'BALANCE_SHEET'
CASH_FLOW = 'CASH_FLOW'
LISTING_STATUS = 'LISTING_STATUS'
EARNINGS_CALENDAR = 'EARNINGS_CALENDAR'
IPO_CALENDAR = 'IPO_CALENDAR'
CURRENCY_EXCHANGE_RATE = 'CURRENCY_EXCHANGE_RATE'
FX_INTRADAY = 'FX_INTRADAY'
FX_DAILY = 'FX_DAILY'
FX_WEEKLY = 'FX_WEEKLY'
FX_MONTHLY = 'FX_MONTHLY'
CURRENCY_EXCHANGE_RATE = 'CURRENCY_EXCHANGE_RATE'
CRYPTO_RATING = 'CRYPTO_RATING'
CRYPTO_INTRADAY = 'CRYPTO_INTRADAY'
DIGITAL_CURRENCY_DAILY = 'DIGITAL_CURRENCY_DAILY'
DIGITAL_CURRENCY_WEEKLY = 'DIGITAL_CURRENCY_WEEKLY'
DIGITAL_CURRENCY_MONTHLY = 'DIGITAL_CURRENCY_MONTHLY'

# ///////////////////////////////////////////////////////////
# TIME SERIES FOR STOCK DATA
# ///////////////////////////////////////////////////////////
ONEMIN = 'Time Series (1min)'
FIVEMIN = 'Time Series (5min)'
FIVETEENMIN = 'Time Series (15min)'
THIRTYMIN = 'Time Series (30min)'
HOUR = 'Time Series (60min)'
FIVEMIN = 'Time Series (5min)'
DAILY = 'Time Series (Daily)' 
WEEKLY = 'Weekly Time Series'
WEEKLY_ADJ = 'Weekly Adjusted Time Series'
MONTLY = 'Monthly Time Series'
MONTLY_ADJ = 'Monthly Adjusted Time Series'
# /////////////////////////////////////////////////////////
# query fundamental stock data of selected type ///////////
# /////////////////////////////////////////////////////////
def query_fundamental_data(func, symbol, apikey, outputsize='full', datatype='json'):
    data = {
        "function": func,
        "symbol": symbol,
        "outputsize": outputsize, # (full) and (compact) are accepted
        "datatype": datatype,
        "apikey": apikey
    }
    return requests.get("https://www.alphavantage.co/query", data).json()

def get_fundamental_dataframe(json):
    df = pd.DataFrame(json['annualReports'])
    df.set_index('fiscalDateEnding', inplace=True)
    return df

# FOR EARNINGS
def get_fundamental_dataframe1(json):
    df = pd.DataFrame(json['annualEarnings'])
    df.set_index('fiscalDateEnding', inplace=True)
    return df

#For Overview
def get_fundamental_dataframe2(json):
    df = pd.DataFrame(json, index=[1])
#     df.set_index(inplace=True)
    return df

def query_timeseries_data(func, symbol, apikey, outputsize='full', datatype='json'):
    data = {
        "function": func,
        "symbol": symbol,
        "outputsize": outputsize, # (full) and (compact) are accepted
        "datatype": datatype,
        "apikey": apikey
    }
    return requests.get("https://www.alphavantage.co/query", data).json()

def get_timeseries_dataframe(json, type):
    df = pd.DataFrame.from_dict(json[type], orient= 'index')
    df.index =  pd.to_datetime(df.index, format='%Y-%m-%d')
    
    df = df.rename(columns={ '1. open': 'Open', '2. high': 'High', '3. low': 'Low', '4. close': 'Close', '5. volume': 'Volume'})
    df = df.astype({'Open': 'float64', 'High': 'float64', 'Low': 'float64','Close': 'float64','Volume': 'float64',})
    df = df[[ 'Open', 'High', 'Low', 'Close', 'Volume']]
    return df

def get_data(stock, apikey):
#     import time

    response_json = query_fundamental_data(INCOME_STATEMENT, stock, apikey)
    INCOME_DATA = get_fundamental_dataframe(response_json)
    time.sleep(12)

    response_json = query_fundamental_data(BALANCE_SHEET, stock, apikey)
    BALANCE_DATA = get_fundamental_dataframe(response_json)
    time.sleep(12)

    response_json = query_fundamental_data(CASH_FLOW, stock, apikey)
    CASHFLOW_DATA = get_fundamental_dataframe(response_json)
    time.sleep(12)

    response_json = query_fundamental_data(EARNINGS, stock, apikey)
    EARNINGS_DATA = get_fundamental_dataframe1(response_json)
    time.sleep(12)

    response_json = query_fundamental_data(OVERVIEW, stock, apikey)
    OVERVIEW_DATA = get_fundamental_dataframe2(response_json)
    time.sleep(12)

    response_json = query_timeseries_data(TIME_SERIES_DAILY, stock, apikey)
    STOCKPRICE_DATA = get_timeseries_dataframe(response_json, DAILY)
    time.sleep(12)
    
    return INCOME_DATA,BALANCE_DATA,CASHFLOW_DATA,OVERVIEW_DATA,EARNINGS_DATA,STOCKPRICE_DATA
# response_json1,response_json2,response_json3,response_json4
# ,INCOME_DATA,BALANCE_DATA,CASHFLOW_DATA,EARNINGS_DATA,OVERVIEW_DATA


In [3]:
# Final 
def extractdata(stock,EarningorRevenue):
    # EarningorRevenue = [0] => Earning Estimate
    # EarningorRevenue = [1] => Revenue Estimate
    headers = {'User-Agent': 'Mozilla/5.0'}

    url = f"https://finance.yahoo.com/quote/{stock}/analysis?p={stock}"

    r = requests.get(url, headers=headers)
    # print(r.text)

    soup = BeautifulSoup(r.text, 'html.parser')
    # print(soup.prettify())
    Estimate_Table = soup.find_all("table", attrs={"class": "W(100%) M(0) BdB Bdc($seperatorColor) Mb(25px)"})
    table_thead = Estimate_Table[EarningorRevenue].thead.find_all("th")
    table_body = Estimate_Table[EarningorRevenue].find("tbody")

    headingitem = []
    table_data = []
    for i in table_thead:
    #     print(i.text)
        headingitem.append(i.text)
    for body in table_body:
        row_list = []
        for row in body:
            for item in row:
                row_list.append(item.text)        
        table_data.append(row_list)
    df = pd.DataFrame(table_data, columns = [headingitem])
    return df

In [4]:
def findcolnamematch(df,nameincluded):
#     nameincluded = 'income'
    cols=[]
    for i in df.columns.to_list():
        x = i.lower()
        if nameincluded in x:
            cols.append(i)
    return cols
# cols = findcolnamematch(IBC,'shares')
# IBC[cols]

In [5]:
def changeMoneytoNumbers(df,targetRow,colnum = 3):
    m = {'K': 3, 'M': 6, 'B': 9, 'T': 12}
    col = Revenue_Estimate.columns.to_list()[colnum]
    firstcol = df.columns.to_list()[0]
    df = df[df[firstcol] == targetRow]
    df[col] = [int(float(i[:-1]) * 10 ** m[i[-1]]) for i in df[col]]
    return(df)

In [6]:
def totalrevenueExtension(df,rownum):
    df['totalRevenue'].iloc[rownum] = ( df['totalRevenue'].iloc[rownum+1]*(100 +  df['totalRevenue_Growth_Rate_Mean'].iloc[rownum+1]))/100
    return df

In [7]:
def obtaindata(stock):
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    #######################################
    # beta and market cap
    url_beta_marketCap = f'https://finance.yahoo.com/quote/{stock}?p={stock}'
    r_beta_marketCap = requests.get(url_beta_marketCap, headers=headers)
    soup_beta_marketCap = BeautifulSoup(r_beta_marketCap.text, 'html.parser')
    table = soup_beta_marketCap.find("table", attrs={"class": "W(100%) M(0) Bdcl(c)"})
    dataitem = table.tbody.find_all("tr")
    emptylist = []
    for i in dataitem[:2]:
        for j in i:
            emptylist.append(j.text) 
    marketcap = emptylist[1]
    beta = emptylist[3]
    
    ####################################################
    # To get bond 
    url_bond = f'https://finance.yahoo.com/bonds'
    r_bond = requests.get(url_bond, headers=headers)
    soup_bond = BeautifulSoup(r_bond.text, 'html.parser')
    table_bond = soup_bond.find("table", attrs={"class": "W(100%)"})
    treasurybond = []
    for j in table_bond.tbody.find_all("tr")[2]:
        treasurybond.append(j.text)
    bondvalue = treasurybond[2]
    
    return stock, marketcap,beta,bondvalue

# Final

In [10]:
# Final

stocks = ['MSFT']
API = ['53K56OJAX5T2DQJZ']
for x,y in zip(stocks,API):
    print(x,y)
    INCOME_DATA,BALANCE_DATA,CASHFLOW_DATA,OVERVIEW_DATA,EARNINGS_DATA,STOCKPRICE_DATA = get_data(x,y)

    BALANCE_DATA = BALANCE_DATA.drop(['reportedCurrency'], axis = 1)
    CASHFLOW_DATA = CASHFLOW_DATA.drop(['reportedCurrency','netIncome'], axis = 1)

    # Define percentage change formula for EARNING data
    EARNINGS_DATA = EARNINGS_DATA.apply(pd.to_numeric)
    EARNINGS_DATA['shifted_EPS'] = EARNINGS_DATA.reportedEPS.shift(-1, axis = 0)
    EARNINGS_DATA['EPS_Growth_Rate']= 100*((EARNINGS_DATA['reportedEPS'] - EARNINGS_DATA['shifted_EPS'])/ abs(EARNINGS_DATA['shifted_EPS']))

    EARNINGS_DATA[['EPS_Growth_Avg_Perc']] = EARNINGS_DATA[:6]['EPS_Growth_Rate'].mean()
    IBC = pd.merge(pd.merge(pd.merge(INCOME_DATA,BALANCE_DATA,on='fiscalDateEnding')
                            ,CASHFLOW_DATA,on='fiscalDateEnding')
                   ,EARNINGS_DATA,on='fiscalDateEnding')
    IBC = IBC.reset_index(0).reset_index(drop=True)
    IBC.replace('None', value=0, inplace=True)

    # # To replace change type to int after the column of 'reportedcurrency'
    index_no = IBC.columns.get_loc('reportedCurrency')
    index_no_starting = index_no+1
    IBC.iloc[:,index_no_starting:] = IBC.iloc[:,index_no_starting:].apply(pd.to_numeric)


    IBC['TotalDebt_FOR_DERatio'] = IBC['shortTermDebt']+IBC['longTermDebt']
    IBC['D/E Ratio'] = IBC['TotalDebt_FOR_DERatio']/IBC['totalShareholderEquity']

    IBC1 = IBC.copy()
    IBC1= pd.concat([OVERVIEW_DATA.reset_index(drop=True),IBC1.iloc[:1,:]], axis=1)
    IBC1['EntryPrice'] = IBC1['EPS_Growth_Avg_Perc'] * float(IBC1['EPS'])



    price = STOCKPRICE_DATA.iloc[:1,:].reset_index(drop=True)    
    IBC1 = pd.concat([IBC1,price], axis=1)

    IBC1['new_EntryPrice'] = [-x if y <0 else x for x,y in zip(IBC1['EntryPrice'],IBC1['EPS_Growth_Avg_Perc'])]
    
    # To obtain Analyst Predicted Revenue
    Revenue_Estimate = extractdata(x,1)
    targetRow = 'Avg. Estimate'

    Revenue_Estimate = changeMoneytoNumbers(Revenue_Estimate,targetRow,colnum = 3)
    Revenue_Estimate = changeMoneytoNumbers(Revenue_Estimate,targetRow,colnum = 4)
    analystval = Revenue_Estimate.iloc[:,-2:].values.tolist()[0]
    analystval[0], analystval[1] = analystval[1], analystval[0]
    
    IBC_DCF = IBC.copy()

    col = ['fiscalDateEnding','totalRevenue','netIncome','operatingCashflow','capitalExpenditures','interestAndDebtExpense','currentLongTermDebt','currentDebt',
       'longTermDebtNoncurrent','incomeBeforeTax','incomeTaxExpense','commonStockSharesOutstanding']
    IBC_DCF1 = IBC_DCF[col]
    IBC_DCF1['FreeCashFlowToEquity'] = IBC_DCF1['operatingCashflow'] - IBC_DCF1['capitalExpenditures']
    IBC_DCF1['FCFE/NetIncome(100%)'] = (IBC_DCF1['FreeCashFlowToEquity'] / IBC_DCF1['netIncome'])*100
    IBC_DCF1['Lowest_FCFE/NetIncome(100%)'] = IBC_DCF1['FCFE/NetIncome(100%)'].min()

    #create 2 years of blank data
    listofna = (len(IBC_DCF1.columns)-1)*[0]
    date = ['2025-01-22','2024-01-22','2023-01-22','2022-01-22']
    row=[]
    for i in date:
        onedate = []
        onedate.append(i)
        for j in listofna:
            onedate.append(j)
        row.append(onedate)

    # putting blank row into the df    
    colss = IBC_DCF1.columns.to_list()
    newrow = pd.DataFrame(row,columns = colss).reset_index(drop=True)
    IBC_DCF1 = pd.concat([newrow, IBC_DCF1], ignore_index=True, axis=0)

    # Replace the analyst avg rating into the df
    IBC_DCF1.iloc[2:4,1] = analystval
    
    IBC_DCF2 = IBC_DCF1.copy()
    IBC_DCF2['shifted_revenue'] = IBC_DCF2.totalRevenue.shift(-1, axis = 0)
    IBC_DCF2['totalRevenue_Growth_Rate(100%)']= 100*((IBC_DCF2['totalRevenue'] - IBC_DCF2['shifted_revenue'])/ abs(IBC_DCF2['shifted_revenue']))
    IBC_DCF2['totalRevenue_Growth_Rate(100%)'][1] = 0
    IBC_DCF2['totalRevenue_Growth_Rate_Mean'] = IBC_DCF2['totalRevenue_Growth_Rate(100%)'].mean()
    # IBC_DCF2['totalRevenue'].iloc[1] = ( IBC_DCF2['totalRevenue'].iloc[2]*(100 +  IBC_DCF2['totalRevenue_Growth_Rate_Mean'].iloc[2]))/100
    IBC_DCF2 = totalrevenueExtension(IBC_DCF2,1)
    IBC_DCF2 = totalrevenueExtension(IBC_DCF2,0)
    IBC_DCF2['NetIncomeMargin'] = IBC_DCF2['netIncome'] / IBC_DCF2['totalRevenue']
    IBC_DCF2['netIncome'].iloc[:4] = IBC_DCF2['totalRevenue'].iloc[:4] * IBC_DCF2[IBC_DCF2['NetIncomeMargin']>0]['NetIncomeMargin'].mean()
    IBC_DCF2['FreeCashFlowToEquity'].iloc[:4] = IBC_DCF2['netIncome'].iloc[:4]* (IBC_DCF2[IBC_DCF2['FCFE/NetIncome(100%)']>0]['FCFE/NetIncome(100%)'].min()/100)

    marketcap_beta_bond = obtaindata(x)

    rd = (IBC_DCF2['interestAndDebtExpense'][4]/(IBC_DCF2['currentLongTermDebt'][4]+IBC_DCF2['longTermDebtNoncurrent'][4]))*100
    if np.isnan(rd) ==True:
        rd = 1
    t = IBC_DCF2['incomeTaxExpense'][4]/IBC_DCF2['incomeBeforeTax'][4]

    # Market Cap
    marketcap = marketcap_beta_bond[1]
    m = {'K': 3, 'M': 6, 'B': 9, 'T': 12}
    marketcap = int(float(marketcap[:-1]) * 10 ** m[marketcap[-1]])
    totaldebt = IBC_DCF2['longTermDebtNoncurrent'][4]+IBC_DCF2['currentDebt'][4]
    total_Totaldebt_Marketcap = marketcap+totaldebt
    totaldebtRatio = totaldebt/total_Totaldebt_Marketcap*100
    marketcapRatio = marketcap/total_Totaldebt_Marketcap*100

    # All these are in percentage
    Rm = 10
    B = float(marketcap_beta_bond[2])
    rf = float(marketcap_beta_bond[3])
    re = rf+B*(Rm-rf)

    #WACC
    # wd = totaldebtratio
    # we = marketcapratio
    # WACC = wdrd(1-t) + were
    WACC = ((totaldebtRatio*rd*(1-t)) +(marketcapRatio*re))/100
    #PerpectualGrowth in %
    PerpectualGrowth = 2.5
    g = PerpectualGrowth/100
    r = WACC / 100
    sharesOutstanding = IBC_DCF2['commonStockSharesOutstanding']
    TerminalValue = (IBC_DCF2['FreeCashFlowToEquity'][0]*(1+g))/(r - g)

    discountfactorPower = []
    for i in range(4,0,-1):
    #     print(i)
        discountfactorPower.append((r+1)**i)
    IBC_DCF2['DiscountFactor'] = 0
    IBC_DCF2.loc[:3,'DiscountFactor'] = discountfactorPower
    IBC_DCF2['PVofFCF'] = 0
    IBC_DCF2.loc[:3,'PVofFCF'] = IBC_DCF2.loc[:3,'FreeCashFlowToEquity'] * IBC_DCF2.loc[:3,'DiscountFactor']
    PVofFCF_TerminalValue = TerminalValue * IBC_DCF2['DiscountFactor'][0]
    TodaysValue = PVofFCF_TerminalValue + IBC_DCF2['PVofFCF'][:4].sum()
    DCF_Stockprice = TodaysValue / IBC_DCF2['commonStockSharesOutstanding'][4]
    
    # What we want
    MichaelMethodValuation_Price = IBC1['new_EntryPrice'][0]
    
    DCF_Stockprice

MSFT 53K56OJAX5T2DQJZ


C:\Users\sweet\AppData\Local\Temp/ipykernel_18448/1203267060.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = [int(float(i[:-1]) * 10 ** m[i[-1]]) for i in df[col]]
C:\Users\sweet\AppData\Local\Temp/ipykernel_18448/3191591697.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IBC_DCF1['FreeCashFlowToEquity'] = IBC_DCF1['operatingCashflow'] - IBC_DCF1['capitalExpenditures']
C:\Users\sweet\AppData\Local\Temp/ipykernel_18448/3191591697.py:63: SettingWithCopyWarning: 
A value is trying to be set 

# Testing for multiple stock

In [16]:
# Every 83 stocks change 1 api for 500 stocks
APIs = [
        'GYX11ISSPTSUX78K',
        'LYKZ99FFKIROA5CD',
        'TCQ580R24NR326XY',
        'W73KWACZE3ZXBDHV',
        'CKGAJT5WBGB6ZO4R',
        'XM7WRAS9NGUK7K1D',
        '53K56OJAX5T2DQJZ'
]
# GYX11ISSPTSUX78K
# BHUMOY6ID7RODFGB
# stockeg = stocks['Ticker'][:7]
# 500stocks * 6 API call = 3000 api calls [1 api = 500 calls per day]
# Every 83 stocks change 1 api for 500 stocks

apitouse = []
for i in APIs:
    for j in range(80):
        apitouse.append(i)
apitouse

['GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTSUX78K',
 'GYX11ISSPTS

In [82]:
stocks = ['MSFT','AAPL','SAM']
API = apitouse
# API = ['53K56OJAX5T2DQJZ']

In [83]:
# TESTING 
# dfstock = pd.Dataframe(columns = cols)
fullwantedlist = []
nooblist = []
# stocks = ['MSFT']
# API = ['53K56OJAX5T2DQJZ']
for x,y in zip(stocks,API):
    print(x,y)
    try:
        INCOME_DATA,BALANCE_DATA,CASHFLOW_DATA,OVERVIEW_DATA,EARNINGS_DATA,STOCKPRICE_DATA = get_data(x,y)

        BALANCE_DATA = BALANCE_DATA.drop(['reportedCurrency'], axis = 1)
        CASHFLOW_DATA = CASHFLOW_DATA.drop(['reportedCurrency','netIncome'], axis = 1)

        # Define percentage change formula for EARNING data
        EARNINGS_DATA = EARNINGS_DATA.apply(pd.to_numeric)
        EARNINGS_DATA['shifted_EPS'] = EARNINGS_DATA.reportedEPS.shift(-1, axis = 0)
        EARNINGS_DATA['EPS_Growth_Rate']= 100*((EARNINGS_DATA['reportedEPS'] - EARNINGS_DATA['shifted_EPS'])/ abs(EARNINGS_DATA['shifted_EPS']))

        EARNINGS_DATA[['EPS_Growth_Avg_Perc']] = EARNINGS_DATA[:6]['EPS_Growth_Rate'].mean()
        IBC = pd.merge(pd.merge(pd.merge(INCOME_DATA,BALANCE_DATA,on='fiscalDateEnding')
                                ,CASHFLOW_DATA,on='fiscalDateEnding')
                       ,EARNINGS_DATA,on='fiscalDateEnding')
        IBC = IBC.reset_index(0).reset_index(drop=True)
        IBC.replace('None', value=0, inplace=True)

        # # To replace change type to int after the column of 'reportedcurrency'
        index_no = IBC.columns.get_loc('reportedCurrency')
        index_no_starting = index_no+1
        IBC.iloc[:,index_no_starting:] = IBC.iloc[:,index_no_starting:].apply(pd.to_numeric)


        IBC['TotalDebt_FOR_DERatio'] = IBC['shortTermDebt']+IBC['longTermDebt']
        IBC['D/E Ratio'] = IBC['TotalDebt_FOR_DERatio']/IBC['totalShareholderEquity']

        IBC1 = IBC.copy()
        IBC1= pd.concat([OVERVIEW_DATA.reset_index(drop=True),IBC1.iloc[:1,:]], axis=1)
        IBC1['EntryPrice'] = IBC1['EPS_Growth_Avg_Perc'] * float(IBC1['EPS'])



        price = STOCKPRICE_DATA.iloc[:1,:].reset_index(drop=True)    
        IBC1 = pd.concat([IBC1,price], axis=1)

        IBC1['new_EntryPrice'] = [-x if y <0 else x for x,y in zip(IBC1['EntryPrice'],IBC1['EPS_Growth_Avg_Perc'])]

        # To obtain Analyst Predicted Revenue
        Revenue_Estimate = extractdata(x,1)
        targetRow = 'Avg. Estimate'

        Revenue_Estimate = changeMoneytoNumbers(Revenue_Estimate,targetRow,colnum = 3)
        Revenue_Estimate = changeMoneytoNumbers(Revenue_Estimate,targetRow,colnum = 4)
        analystval = Revenue_Estimate.iloc[:,-2:].values.tolist()[0]
        analystval[0], analystval[1] = analystval[1], analystval[0]

        IBC_DCF = IBC.copy()

        col = ['fiscalDateEnding','totalRevenue','netIncome','operatingCashflow','capitalExpenditures','interestAndDebtExpense','currentLongTermDebt','currentDebt',
           'longTermDebtNoncurrent','incomeBeforeTax','incomeTaxExpense','commonStockSharesOutstanding']
        IBC_DCF1 = IBC_DCF[col]
        IBC_DCF1['FreeCashFlowToEquity'] = IBC_DCF1['operatingCashflow'] - IBC_DCF1['capitalExpenditures']
        IBC_DCF1['FCFE/NetIncome(100%)'] = (IBC_DCF1['FreeCashFlowToEquity'] / IBC_DCF1['netIncome'])*100
        IBC_DCF1['Lowest_FCFE/NetIncome(100%)'] = IBC_DCF1['FCFE/NetIncome(100%)'].min()

        #create 2 years of blank data
        listofna = (len(IBC_DCF1.columns)-1)*[0]
        date = ['2025-01-22','2024-01-22','2023-01-22','2022-01-22']
        row=[]
        for i in date:
            onedate = []
            onedate.append(i)
            for j in listofna:
                onedate.append(j)
            row.append(onedate)

        # putting blank row into the df    
        colss = IBC_DCF1.columns.to_list()
        newrow = pd.DataFrame(row,columns = colss).reset_index(drop=True)
        IBC_DCF1 = pd.concat([newrow, IBC_DCF1], ignore_index=True, axis=0)

        # Replace the analyst avg rating into the df
        IBC_DCF1.iloc[2:4,1] = analystval

        IBC_DCF2 = IBC_DCF1.copy()
        IBC_DCF2['shifted_revenue'] = IBC_DCF2.totalRevenue.shift(-1, axis = 0)
        IBC_DCF2['totalRevenue_Growth_Rate(100%)']= 100*((IBC_DCF2['totalRevenue'] - IBC_DCF2['shifted_revenue'])/ abs(IBC_DCF2['shifted_revenue']))
        IBC_DCF2['totalRevenue_Growth_Rate(100%)'][1] = 0
        IBC_DCF2['totalRevenue_Growth_Rate_Mean'] = IBC_DCF2['totalRevenue_Growth_Rate(100%)'].mean()
        # IBC_DCF2['totalRevenue'].iloc[1] = ( IBC_DCF2['totalRevenue'].iloc[2]*(100 +  IBC_DCF2['totalRevenue_Growth_Rate_Mean'].iloc[2]))/100
        IBC_DCF2 = totalrevenueExtension(IBC_DCF2,1)
        IBC_DCF2 = totalrevenueExtension(IBC_DCF2,0)
        IBC_DCF2['NetIncomeMargin'] = IBC_DCF2['netIncome'] / IBC_DCF2['totalRevenue']
        IBC_DCF2['netIncome'].iloc[:4] = IBC_DCF2['totalRevenue'].iloc[:4] * IBC_DCF2[IBC_DCF2['NetIncomeMargin']>0]['NetIncomeMargin'].mean()
        IBC_DCF2['FreeCashFlowToEquity'].iloc[:4] = IBC_DCF2['netIncome'].iloc[:4]* (IBC_DCF2[IBC_DCF2['FCFE/NetIncome(100%)']>0]['FCFE/NetIncome(100%)'].min()/100)

        marketcap_beta_bond = obtaindata(x)

        rd = (IBC_DCF2['interestAndDebtExpense'][4]/(IBC_DCF2['currentLongTermDebt'][4]+IBC_DCF2['longTermDebtNoncurrent'][4]))*100
        if np.isnan(rd) ==True:
            rd = 1
        t = IBC_DCF2['incomeTaxExpense'][4]/IBC_DCF2['incomeBeforeTax'][4]

        # Market Cap
        marketcap = marketcap_beta_bond[1]
        m = {'K': 3, 'M': 6, 'B': 9, 'T': 12}
        marketcap = int(float(marketcap[:-1]) * 10 ** m[marketcap[-1]])
        totaldebt = IBC_DCF2['longTermDebtNoncurrent'][4]+IBC_DCF2['currentDebt'][4]
        total_Totaldebt_Marketcap = marketcap+totaldebt
        totaldebtRatio = totaldebt/total_Totaldebt_Marketcap*100
        marketcapRatio = marketcap/total_Totaldebt_Marketcap*100

        # All these are in percentage
        Rm = 10
        B = float(marketcap_beta_bond[2])
        rf = float(marketcap_beta_bond[3])
        re = rf+B*(Rm-rf)

        #WACC
        # wd = totaldebtratio
        # we = marketcapratio
        # WACC = wdrd(1-t) + were
        WACC = ((totaldebtRatio*rd*(1-t)) +(marketcapRatio*re))/100
        #PerpectualGrowth in %
        PerpectualGrowth = 2.5
        g = PerpectualGrowth/100
        r = WACC / 100
        sharesOutstanding = IBC_DCF2['commonStockSharesOutstanding']
        TerminalValue = (IBC_DCF2['FreeCashFlowToEquity'][0]*(1+g))/(r - g)

        discountfactorPower = []
        for i in range(4,0,-1):
        #     print(i)
            discountfactorPower.append((r+1)**i)
        IBC_DCF2['DiscountFactor'] = 0
        IBC_DCF2.loc[:3,'DiscountFactor'] = discountfactorPower
        IBC_DCF2['PVofFCF'] = 0
        IBC_DCF2.loc[:3,'PVofFCF'] = IBC_DCF2.loc[:3,'FreeCashFlowToEquity'] * IBC_DCF2.loc[:3,'DiscountFactor']
        PVofFCF_TerminalValue = TerminalValue * IBC_DCF2['DiscountFactor'][0]
        TodaysValue = PVofFCF_TerminalValue + IBC_DCF2['PVofFCF'][:4].sum()
        DCF_Stockprice = TodaysValue / IBC_DCF2['commonStockSharesOutstanding'][4]

        # What we want
        currentstockprice = STOCKPRICE_DATA['Close'][0]
        MichaelMethodValuation_Price = IBC1['new_EntryPrice'][0]

        wantedlist = [x,DCF_Stockprice,MichaelMethodValuation_Price,currentstockprice]
        fullwantedlist.append(wantedlist)
    except:
        nooblist.apend(x)
        continue

colsssssss = ['stock','DCF_Stockprice','MichaelMethodValuation_Price','currentstockprice']
pd.DataFrame(fullwantedlist, columns=colsssssss)


MSFT GYX11ISSPTSUX78K


C:\Users\sweet\AppData\Local\Temp/ipykernel_18448/1203267060.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = [int(float(i[:-1]) * 10 ** m[i[-1]]) for i in df[col]]
C:\Users\sweet\AppData\Local\Temp/ipykernel_18448/2609556998.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IBC_DCF1['FreeCashFlowToEquity'] = IBC_DCF1['operatingCashflow'] - IBC_DCF1['capitalExpenditures']
C:\Users\sweet\AppData\Local\Temp/ipykernel_18448/2609556998.py:62: SettingWithCopyWarning: 
A value is trying to be set 

AAPL GYX11ISSPTSUX78K
SAM GYX11ISSPTSUX78K


C:\Users\sweet\AppData\Local\Temp/ipykernel_18448/2609556998.py:98: RuntimeWarning: invalid value encountered in longlong_scalars
  rd = (IBC_DCF2['interestAndDebtExpense'][4]/(IBC_DCF2['currentLongTermDebt'][4]+IBC_DCF2['longTermDebtNoncurrent'][4]))*100


,stock,DCF_Stockprice,MichaelMethodValuation_Price,currentstockprice
0,MSFT,270.889194,71.093703,318.27
1,AAPL,144.774224,20.859465,175.53
2,SAM,480.065486,91.933301,485.00


# Read betabond list 

In [85]:
import json
with open('listofStockBondBeta.txt', 'r') as f:
    listofStockBondBeta = json.loads(f.read())

In [86]:
listofStockBondBeta

[['A', '45.152B', '1.00', '1.7310'],
 ['AAL', '11.979B', '1.66', '1.7310'],
 ['AAP', '14.811B', '1.27', '1.7310'],
 ['AAPL', '2.867T', '1.20', '1.7310'],
 ['ABBV', '242.712B', '0.82', '1.7310'],
 ['ABC', '28.246B', '0.51', '1.7310'],
 ['ABMD', '14.351B', '1.29', '1.7310'],
 ['ABT', '236.456B', '0.70', '1.7310'],
 ['ACN', '238.576B', '1.16', '1.7310'],
 ['ADBE', '253.302B', '1.00', '1.7310'],
 ['ADI', '91.276B', '1.11', '1.7310'],
 ['ADM', '39.396B', '0.90', '1.7310'],
 ['ADP', '98.869B', '0.76', '1.7310'],
 ['ADSK', '57.719B', '1.40', '1.7800'],
 ['AEE', '22.595B', '0.34', '1.7800'],
 ['AEP', '44.923B', '0.35', '1.7800'],
 ['AES', '15.308B', 'N/A', '1.7800'],
 ['AFL', '41.617B', '0.99', '1.7800'],
 ['AIG', '50.989B', '1.38', '1.7800'],
 ['AIV', '1.093B', 'N/A', '1.7800'],
 ['AIZ', '8.908B', '0.64', '1.7800'],
 ['AJG', '33.668B', '0.69', '1.7800'],
 ['AKAM', '18.303B', '0.54', '1.7800'],
 ['ALB', '26.55B', '1.54', '1.7800'],
 ['ALGN', '43.283B', '1.57', '1.7800'],
 ['ALK', '6.905B', '1.

# Single Stock Inspection


In [63]:
stocks = ['SAM']
API = ['53K56OJAX5T2DQJZ']
for x,y in zip(stocks,API):
    print(x,y)
    INCOME_DATA,BALANCE_DATA,CASHFLOW_DATA,OVERVIEW_DATA,EARNINGS_DATA,STOCKPRICE_DATA = get_data(x,y)

    BALANCE_DATA = BALANCE_DATA.drop(['reportedCurrency'], axis = 1)
    CASHFLOW_DATA = CASHFLOW_DATA.drop(['reportedCurrency','netIncome'], axis = 1)

    # Define percentage change formula for EARNING data
    EARNINGS_DATA = EARNINGS_DATA.apply(pd.to_numeric)
    EARNINGS_DATA['shifted_EPS'] = EARNINGS_DATA.reportedEPS.shift(-1, axis = 0)
    EARNINGS_DATA['EPS_Growth_Rate']= 100*((EARNINGS_DATA['reportedEPS'] - EARNINGS_DATA['shifted_EPS'])/ abs(EARNINGS_DATA['shifted_EPS']))

    EARNINGS_DATA[['EPS_Growth_Avg_Perc']] = EARNINGS_DATA[:6]['EPS_Growth_Rate'].mean()
    IBC = pd.merge(pd.merge(pd.merge(INCOME_DATA,BALANCE_DATA,on='fiscalDateEnding')
                            ,CASHFLOW_DATA,on='fiscalDateEnding')
                   ,EARNINGS_DATA,on='fiscalDateEnding')
    IBC = IBC.reset_index(0).reset_index(drop=True)
    IBC.replace('None', value=0, inplace=True)

    # # To replace change type to int after the column of 'reportedcurrency'
    index_no = IBC.columns.get_loc('reportedCurrency')
    index_no_starting = index_no+1
    IBC.iloc[:,index_no_starting:] = IBC.iloc[:,index_no_starting:].apply(pd.to_numeric)


    IBC['TotalDebt_FOR_DERatio'] = IBC['shortTermDebt']+IBC['longTermDebt']
    IBC['D/E Ratio'] = IBC['TotalDebt_FOR_DERatio']/IBC['totalShareholderEquity']

    IBC1 = IBC.copy()
    IBC1= pd.concat([OVERVIEW_DATA.reset_index(drop=True),IBC1.iloc[:1,:]], axis=1)
    IBC1['EntryPrice'] = IBC1['EPS_Growth_Avg_Perc'] * float(IBC1['EPS'])



    price = STOCKPRICE_DATA.iloc[:1,:].reset_index(drop=True)    
    IBC1 = pd.concat([IBC1,price], axis=1)

    IBC1['new_EntryPrice'] = [-x if y <0 else x for x,y in zip(IBC1['EntryPrice'],IBC1['EPS_Growth_Avg_Perc'])]
    
    # To obtain Analyst Predicted Revenue
    Revenue_Estimate = extractdata(x,1)
    targetRow = 'Avg. Estimate'

    Revenue_Estimate = changeMoneytoNumbers(Revenue_Estimate,targetRow,colnum = 3)
    Revenue_Estimate = changeMoneytoNumbers(Revenue_Estimate,targetRow,colnum = 4)
    analystval = Revenue_Estimate.iloc[:,-2:].values.tolist()[0]
    analystval[0], analystval[1] = analystval[1], analystval[0]
    
    IBC_DCF = IBC.copy()

    col = ['fiscalDateEnding','totalRevenue','netIncome','operatingCashflow','capitalExpenditures','interestAndDebtExpense','currentLongTermDebt','currentDebt',
       'longTermDebtNoncurrent','incomeBeforeTax','incomeTaxExpense','commonStockSharesOutstanding']
    IBC_DCF1 = IBC_DCF[col]
    IBC_DCF1['FreeCashFlowToEquity'] = IBC_DCF1['operatingCashflow'] - IBC_DCF1['capitalExpenditures']
    IBC_DCF1['FCFE/NetIncome(100%)'] = (IBC_DCF1['FreeCashFlowToEquity'] / IBC_DCF1['netIncome'])*100
    IBC_DCF1['Lowest_FCFE/NetIncome(100%)'] = IBC_DCF1['FCFE/NetIncome(100%)'].min()

    #create 2 years of blank data
    listofna = (len(IBC_DCF1.columns)-1)*[0]
    date = ['2025-01-22','2024-01-22','2023-01-22','2022-01-22']
    row=[]
    for i in date:
        onedate = []
        onedate.append(i)
        for j in listofna:
            onedate.append(j)
        row.append(onedate)

    # putting blank row into the df    
    colss = IBC_DCF1.columns.to_list()
    newrow = pd.DataFrame(row,columns = colss).reset_index(drop=True)
    IBC_DCF1 = pd.concat([newrow, IBC_DCF1], ignore_index=True, axis=0)

    # Replace the analyst avg rating into the df
    IBC_DCF1.iloc[2:4,1] = analystval
    
    IBC_DCF2 = IBC_DCF1.copy()
    IBC_DCF2['shifted_revenue'] = IBC_DCF2.totalRevenue.shift(-1, axis = 0)
    IBC_DCF2['totalRevenue_Growth_Rate(100%)']= 100*((IBC_DCF2['totalRevenue'] - IBC_DCF2['shifted_revenue'])/ abs(IBC_DCF2['shifted_revenue']))
    IBC_DCF2['totalRevenue_Growth_Rate(100%)'][1] = 0
    IBC_DCF2['totalRevenue_Growth_Rate_Mean'] = IBC_DCF2['totalRevenue_Growth_Rate(100%)'].mean()
    # IBC_DCF2['totalRevenue'].iloc[1] = ( IBC_DCF2['totalRevenue'].iloc[2]*(100 +  IBC_DCF2['totalRevenue_Growth_Rate_Mean'].iloc[2]))/100
    IBC_DCF2 = totalrevenueExtension(IBC_DCF2,1)
    IBC_DCF2 = totalrevenueExtension(IBC_DCF2,0)
    IBC_DCF2['NetIncomeMargin'] = IBC_DCF2['netIncome'] / IBC_DCF2['totalRevenue']
    IBC_DCF2['netIncome'].iloc[:4] = IBC_DCF2['totalRevenue'].iloc[:4] * IBC_DCF2[IBC_DCF2['NetIncomeMargin']>0]['NetIncomeMargin'].mean()
    # if optimistic can change the min to mean function
    IBC_DCF2['FreeCashFlowToEquity'].iloc[:4] = IBC_DCF2['netIncome'].iloc[:4]* (IBC_DCF2[IBC_DCF2['FCFE/NetIncome(100%)']>0]['FCFE/NetIncome(100%)'].min()/100)

    marketcap_beta_bond = obtaindata(x)

    rd = (IBC_DCF2['interestAndDebtExpense'][4]/(IBC_DCF2['currentLongTermDebt'][4]+IBC_DCF2['longTermDebtNoncurrent'][4]))*100
    if np.isnan(rd) ==True:
        rd = 1
        
    t = IBC_DCF2['incomeTaxExpense'][4]/IBC_DCF2['incomeBeforeTax'][4]

    # Market Cap
    marketcap = marketcap_beta_bond[1]
    m = {'K': 3, 'M': 6, 'B': 9, 'T': 12}
    marketcap = int(float(marketcap[:-1]) * 10 ** m[marketcap[-1]])
    totaldebt = IBC_DCF2['longTermDebtNoncurrent'][4]+IBC_DCF2['currentDebt'][4]
    total_Totaldebt_Marketcap = marketcap+totaldebt
    totaldebtRatio = totaldebt/total_Totaldebt_Marketcap*100
    marketcapRatio = marketcap/total_Totaldebt_Marketcap*100

    # All these are in percentage
    Rm = 10
    B = float(marketcap_beta_bond[2])
    rf = float(marketcap_beta_bond[3])
    re = rf+B*(Rm-rf)

    #WACC
    # wd = totaldebtratio
    # we = marketcapratio
    # WACC = wdrd(1-t) + were
    WACC = ((totaldebtRatio*rd*(1-t)) +(marketcapRatio*re))/100
    #PerpectualGrowth in %
    PerpectualGrowth = 2.5
    g = PerpectualGrowth/100
    r = WACC / 100
    sharesOutstanding = IBC_DCF2['commonStockSharesOutstanding']
    TerminalValue = (IBC_DCF2['FreeCashFlowToEquity'][0]*(1+g))/(r - g)

    discountfactorPower = []
    for i in range(4,0,-1):
    #     print(i)
        discountfactorPower.append((r+1)**i)
    IBC_DCF2['DiscountFactor'] = 0
    IBC_DCF2.loc[:3,'DiscountFactor'] = discountfactorPower
    IBC_DCF2['PVofFCF'] = 0
    IBC_DCF2.loc[:3,'PVofFCF'] = IBC_DCF2.loc[:3,'FreeCashFlowToEquity'] * IBC_DCF2.loc[:3,'DiscountFactor']
    PVofFCF_TerminalValue = TerminalValue * IBC_DCF2['DiscountFactor'][0]
    TodaysValue = PVofFCF_TerminalValue + IBC_DCF2['PVofFCF'][:4].sum()
    DCF_Stockprice = TodaysValue / IBC_DCF2['commonStockSharesOutstanding'][4]
    
    # What we want
    MichaelMethodValuation_Price = IBC1['new_EntryPrice'][0]
    
    wantedlist = [x,DCF_Stockprice,MichaelMethodValuation_Price]
#     fullwantedlist.append(wantedlist)

SAM 53K56OJAX5T2DQJZ


C:\Users\sweet\AppData\Local\Temp/ipykernel_18448/1203267060.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = [int(float(i[:-1]) * 10 ** m[i[-1]]) for i in df[col]]
C:\Users\sweet\AppData\Local\Temp/ipykernel_18448/460112742.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IBC_DCF1['FreeCashFlowToEquity'] = IBC_DCF1['operatingCashflow'] - IBC_DCF1['capitalExpenditures']
C:\Users\sweet\AppData\Local\Temp/ipykernel_18448/460112742.py:57: SettingWithCopyWarning: 
A value is trying to be set on

In [64]:
wantedlist

['SAM', 480.1897244823067, 91.93330066073362]